In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

In [2]:
data = pd.read_csv("si630w22-hw3-data.csv")
data

,question_id,question_text,reply_id,reply_text,rlen
0,t3_n27vu3,What's something nice you like to do just to b...,gwhrhmf,Give compliments. It’s extremely easy to do an...,205
1,t3_n2az7m,So what is the best headphones for people who ...,gwiatps,I prefer Raycon Performance Ear Buds. They are...,178
2,t3_n2dzr9,How do you go on knowing a loved one only has ...,gwit1wj,Make it as memorable as the rest of your time ...,278
3,t3_n2iy9q,You’ve been dropped to the year 1800 with all ...,gwjhw8i,They're gonna burn me at the stake for being a...,52
4,t3_n2kuuq,Stuck in bad habits for years and I realized i...,gwkiiie,Using new environments as a way to create heal...,651
...,...,...,...,...,...
5395,t3_njln4h,What is the first thing that comes to mind whe...,gz82i2v,"Austin, we have a problem. Just watched an epi...",168
5396,t3_nkx9c8,What career advice would you give to someone w...,gzfan0b,Work anywhere while you build qualifications a...,449
5397,t3_nm2mc0,What was your most unexplainable and bizarre e...,gzm4krx,Doing my homework at the desk when I was like ...,348
5398,t3_nnbl3i,Do other countries have as much coverage of UF...,gztlr9g,"In Mexico, there’s barely any news if any but ...",377


In [3]:
train = pd.read_csv("si630w22-hw3-train.csv")
train

,id,annotator_id,rating,group
0,t3_n27vu3,user_00,5.0,group_09
1,t3_n27vu3,user_01,5.0,group_09
2,t3_n27vu3,user_02,5.0,group_09
3,t3_n2az7m,user_00,5.0,group_09
4,t3_n2az7m,user_01,5.0,group_09
...,...,...,...,...
17840,t3_np0hd0,user_57,2.0,group_11
17841,t3_np0hd0,user_58,4.0,group_11
17842,t3_np0myc,user_56,1.0,group_11
17843,t3_np0myc,user_57,4.0,group_11


In [4]:
text_list = []
score_list = []
for index, row in data.iterrows():
    s = 0
    cnt = 0
    for score in train.loc[row["question_id"] == train["id"]]["rating"]:
        if not np.isnan(score):
            s += score
            cnt += 1
    if cnt != 0:
        text_list.append(row["question_text"] + " " + row["reply_text"])
        score_list.append(s / cnt)
df = pd.DataFrame({"text": text_list, "rating": score_list}) 
df

,text,rating
0,What's something nice you like to do just to b...,4.40
1,So what is the best headphones for people who ...,4.50
2,How do you go on knowing a loved one only has ...,4.80
3,You’ve been dropped to the year 1800 with all ...,1.40
4,Stuck in bad habits for years and I realized i...,5.00
...,...,...
3774,Is this normal? Should I say something? From a...,4.50
3775,"If Gender is how you feel, and not what your b...",4.25
3776,What is the first thing that comes to mind whe...,4.00
3777,What career advice would you give to someone w...,4.75


In [5]:
text_list_q = []
text_list_a = []
score_list = []
for index, row in data.iterrows():
    s = 0
    cnt = 0
    for score in train.loc[row["question_id"] == train["id"]]["rating"]:
        if not np.isnan(score):
            s += score
            cnt += 1
    if cnt != 0:
        text_list_q.append(row["question_text"])
        text_list_a.append(row["reply_text"])
        score_list.append(s / cnt)
df2 = pd.DataFrame({"text_q": text_list_q, "text_a": text_list_a, "rating": score_list}) 
df2

,text_q,text_a,rating
0,What's something nice you like to do just to b...,Give compliments. It’s extremely easy to do an...,4.40
1,So what is the best headphones for people who ...,I prefer Raycon Performance Ear Buds. They are...,4.50
2,How do you go on knowing a loved one only has ...,Make it as memorable as the rest of your time ...,4.80
3,You’ve been dropped to the year 1800 with all ...,They're gonna burn me at the stake for being a...,1.40
4,Stuck in bad habits for years and I realized i...,Using new environments as a way to create heal...,5.00
...,...,...,...
3774,Is this normal? Should I say something?,"From a guys perspective, this is definitely NO...",4.50
3775,"If Gender is how you feel, and not what your b...",Gender is how you feel. Sometimes it matches w...,4.25
3776,What is the first thing that comes to mind whe...,"Austin, we have a problem. Just watched an epi...",4.00
3777,What career advice would you give to someone w...,Work anywhere while you build qualifications a...,4.75


In [6]:
dev = pd.read_csv("si630w22-hw3-dev.csv")
text_list = []
score_list = []
for index, row in data.iterrows():
    s = 0
    cnt = 0
    for score in dev.loc[row["question_id"] == dev["id"]]["rating"]:
        if not np.isnan(score):
            s += score
            cnt += 1
    if cnt != 0:
        text_list.append(row["question_text"] + " " + row["reply_text"])
        score_list.append(s / cnt)
df_dev = pd.DataFrame({"text": text_list, "rating": score_list}) 
df_dev

,text,rating
0,ELI5: How do analog signals (magnetic tape in ...,4.60
1,"People who carry around a towel with them, why...",1.40
2,"To Architects, How Much Math Is Required In Yo...",3.20
3,What’s touristy thing have you been so excited...,5.00
4,What do you know about Palestine? They have a ...,3.40
...,...,...
806,What simply skill did you learn recently do yo...,2.75
807,What was the most passive-aggressive gift you ...,4.50
808,What is a weird fact that no one really thinks...,3.75
809,What did you forgive a partner for that you wi...,4.75


In [7]:
text_list_q = []
text_list_a = []
score_list = []
for index, row in data.iterrows():
    s = 0
    cnt = 0
    for score in dev.loc[row["question_id"] == dev["id"]]["rating"]:
        if not np.isnan(score):
            s += score
            cnt += 1
    if cnt != 0:
        text_list_q.append(row["question_text"])
        text_list_a.append(row["reply_text"])
        score_list.append(s / cnt)
df_dev2 = pd.DataFrame({"text_q": text_list_q, "text_a": text_list_a, "rating": score_list}) 
df_dev2

,text_q,text_a,rating
0,ELI5: How do analog signals (magnetic tape in ...,It’s varying levels. Think of a photograph on ...,4.60
1,"People who carry around a towel with them, why?","So everyone knows ""that's a frood who knows wh...",1.40
2,"To Architects, How Much Math Is Required In Yo...",For Context: I'm a Year 11 Student (16 y.o) ha...,3.20
3,What’s touristy thing have you been so excited...,"Not a *total* let down, but they don't let you...",5.00
4,What do you know about Palestine?,"They have a just cause, but totally wrong appr...",3.40
...,...,...,...
806,What simply skill did you learn recently do yo...,Grammar and spelling is quite SIMPLE. Would re...,2.75
807,What was the most passive-aggressive gift you ...,"My sister in law, not me. Our father in law an...",4.50
808,What is a weird fact that no one really thinks...,Concept of souls and reincarnation. The has to...,3.75
809,What did you forgive a partner for that you wi...,My hubby got taken in by a Russian bride scam ...,4.75


In [8]:
test = pd.read_csv("si630w22-hw3-test.public.csv")
text_list = []
for index, row in data.iterrows():
    if len(test.loc[row["question_id"] == test["id"]]) != 0:
        text_list.append(row["question_text"] + row["reply_text"])
df_test = pd.DataFrame({"text": text_list}) 
df_test

,text
0,Should I be insecure of my penis size?Truth is...
1,how do you guys lay out for monthly AND weekly...
2,What business could you realistic start with $...
3,Has anyone ever actually been in a classic hor...
4,If you could go back in time and change one th...
...,...
805,What is something you did but never told anyon...
806,How will human civilization end?At it's own hand
807,How do you freshen up in the bathroom every mo...
808,"What was the biggest mistake you've ever made,..."


In [17]:
test = pd.read_csv("si630w22-hw3-test.public.csv")
text_list_q = []
text_list_a = []
id_list = []
for index, row in data.iterrows():
    if len(test.loc[row["question_id"] == test["id"]]) != 0:
        text_list_q.append(row["question_text"])
        text_list_a.append(row["reply_text"])
        id_list.append(row["question_id"])
df_test2 = pd.DataFrame({"id": id_list, "text_q": text_list_q, "text_a": text_list_a}) 
df_test2

,id,text_q,text_a
0,t3_n2ooiu,Should I be insecure of my penis size?,"Truth is, the vast majority of women can't org..."
1,t3_n2to6d,how do you guys lay out for monthly AND weekly...,"I typically only do weekly pages, but the mont..."
2,t3_n2xuk5,What business could you realistic start with $...,"Auto detailing, lawn care my top 2"
3,t3_n2xrc5,Has anyone ever actually been in a classic hor...,"Idk if this counts, but my parents told me tha..."
4,t3_n2yhgh,If you could go back in time and change one th...,I'd tell my 12 year old self about what being ...
...,...,...,...
805,t3_n9w5ix,What is something you did but never told anyone?,"Pretty tame, but: When my older brother was a ..."
806,t3_nf3a7q,How will human civilization end?,At it's own hand
807,t3_npg62d,How do you freshen up in the bathroom every mo...,"I take a look in the mirror and say ""what's up"""
808,t3_nctt1a,"What was the biggest mistake you've ever made,...",I volunteered to DJ at my high school's basket...


In [16]:
# Define pretrained tokenizer and model
model_name = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)

# ----- 1. Preprocess data -----#
# Preprocess data
# X_train = list(df["text"])
# y_train = list(df["rating"])
# X_val = list(df_dev["text"])
# y_val = list(df_dev["rating"])

# X = list(df["text"])
# y = list(df["rating"])
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
# X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized = tokenizer(list(df2["text_q"]), list(df2["text_a"]), padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(list(df_dev2["text_q"]), list(df_dev2["text_a"]), padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters

# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)
#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred, average='micro')
#     precision = precision_score(y_true=labels, y_pred=pred, average='micro')
#     f1 = f1_score(y_true=labels, y_pred=pred, average='micro')
#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# def compute_metrics(p):
#     pred, labels = p
#     pred = pred.reshape((len(pred),))
#     return {"MSE": np.sum((pred - labels) ** 2 / len(pred))}

def compute_metrics(p):
    pred, labels = p
    pred = pred.reshape((len(pred),))
    return {"r": np.corrcoef(pred, labels)[0,1]}


# Define Trainer
args = TrainingArguments(
    output_dir="output_A3",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train pre-trained model
trainer.train()



loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /home/sylva/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json from cache at /home/sylva/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json from cache at /home/sylva/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading fil

Step,Training Loss,Validation Loss,R
500,1.179000,0.759764,0.232935


***** Running Evaluation *****
  Num examples = 811
  Batch size = 8
Saving model checkpoint to output_A3/checkpoint-500
Configuration saved in output_A3/checkpoint-500/config.json
Model weights saved in output_A3/checkpoint-500/pytorch_model.bin


KeyboardInterrupt: 

In [19]:
# ----- 3. Predict -----#
# Load test data
# X_test = list(df_test["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

X_test_tokenized = tokenizer(list(df_test2["text_q"]), list(df_test2["text_a"]),  padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path = "output_A3/checkpoint-1000"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=1)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [20]:
raw_pred

array([[4.5447497],
       [4.241802 ],
       [3.4033468],
       [4.541659 ],
       [3.403532 ],
       [3.4117258],
       [4.5453944],
       [3.4034283],
       [3.7401311],
       [3.403266 ],
       [4.53986  ],
       [3.4126484],
       [3.4033992],
       [3.4030669],
       [4.543202 ],
       [3.4033773],
       [3.403411 ],
       [4.544922 ],
       [3.403387 ],
       [4.5415964],
       [3.4034464],
       [4.532633 ],
       [4.5449224],
       [3.4035335],
       [3.403495 ],
       [4.54333  ],
       [3.4034963],
       [4.510979 ],
       [3.766831 ],
       [4.5444655],
       [4.473453 ],
       [4.264784 ],
       [3.4034588],
       [4.542552 ],
       [3.4034162],
       [4.545183 ],
       [3.4033983],
       [3.4034042],
       [4.545101 ],
       [3.4032319],
       [3.4035265],
       [3.403417 ],
       [3.4033425],
       [3.4033422],
       [4.5391626],
       [3.4033792],
       [4.5434394],
       [3.403455 ],
       [3.4033735],
       [4.515664 ],


## part 3

In [49]:
import tqdm
for i in tqdm.tqdm(range(1, 26, 1)):
    cur_group = "group_"+("%02d" % i)
    train_sub = train.loc[train["group"] != cur_group]
    
    # train set
    text_list = []
    score_list = []
    for index, row in data.iterrows():
        s = 0
        cnt = 0
        for score in train_sub.loc[row["question_id"] == train_sub["id"]]["rating"]:
            if not np.isnan(score):
                s += score
                cnt += 1
        if cnt != 0:
            text_list.append(row["question_text"] + " " + row["reply_text"])
            score_list.append(s / cnt)
    df = pd.DataFrame({"text": text_list, "rating": score_list}) 

    model_name = "microsoft/MiniLM-L12-H384-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)

    X = list(df["text"])
    y = list(df["rating"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)


    class Dataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels=None):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            if self.labels:
                item["labels"] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.encodings["input_ids"])

    train_dataset = Dataset(X_train_tokenized, y_train)
    val_dataset = Dataset(X_val_tokenized, y_val)

    def compute_metrics(p):
        pred, labels = p
        pred = pred.reshape((len(pred),))
        return {"r": np.corrcoef(pred, labels)[0,1]}

    args = TrainingArguments(
        output_dir="out/output"+"_group_"+("%02d" % i),
        evaluation_strategy="steps",
        eval_steps=500,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=1e-4,
        num_train_epochs=2,
        seed=0,
        load_best_model_at_end=True,
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    # Train pre-trained model
    trainer.train()



  0%|          | 0/25 [00:00<?, ?it/s]loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /home/sylva/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json from cache at /home/sylva/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json from cache at /home/sylva/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb

Step,Training Loss,Validation Loss,R
500,0.912100,0.541974,0.548145


***** Running Evaluation *****
  Num examples = 754
  Batch size = 8
Saving model checkpoint to out/output_group_01/checkpoint-500
Configuration saved in out/output_group_01/checkpoint-500/config.json
Model weights saved in out/output_group_01/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from out/output_group_01/checkpoint-500 (score: 0.5419739484786987).
  4%|▍         | 1/25 [02:11<52:25, 131.08s/it]loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /home/sylva/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map

Step,Training Loss,Validation Loss,R
500,0.896000,0.572160,0.573917


***** Running Evaluation *****
  Num examples = 754
  Batch size = 8
Saving model checkpoint to out/output_group_02/checkpoint-500
Configuration saved in out/output_group_02/checkpoint-500/config.json
Model weights saved in out/output_group_02/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from out/output_group_02/checkpoint-500 (score: 0.5721600651741028).
  8%|▊         | 2/25 [04:21<50:03, 130.60s/it]loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /home/sylva/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map

Step,Training Loss,Validation Loss


  8%|▊         | 2/25 [04:47<55:07, 143.82s/it]


KeyboardInterrupt: 

In [57]:
ret = {}
for i in tqdm.tqdm(range(1, 26, 1)):
    cur_group = "group_"+("%02d" % i)
    
    # laod dev dataset
    text_list1 = []
    score_list1 = []
    text_list2 = []
    score_list2 = []
    text_list3 = []
    score_list3 = []
    for index, row in data.iterrows():
        df_subset = dev.loc[row["question_id"] == dev["id"]]
        if cur_group in list(df_subset["group"]):
            s2 = 0
            cnt2 = 0
            s3 = 0
            cnt3 = 0
            for _, j in df_subset.iterrows():
                if not np.isnan(score):
                    if j["group"] == cur_group:
                        # set 2
                        s2 += j["rating"]
                        cnt2 += 1
                    else:
                        # set 3
                        s3 += score
                        cnt3 += 1

            if cnt2 != 0:
                text_list2.append(row["question_text"] + " " + row["reply_text"])
                score_list2.append(s2 / cnt2)
            if cnt3 != 0:
                text_list3.append(row["question_text"] + " " + row["reply_text"])
                score_list3.append(s3 / cnt3)
        else:
            # set1
            s = 0
            cnt = 0
            for score in df_subset["rating"]:
                if not np.isnan(score):
                    s += score
                    cnt += 1
            if cnt != 0:
                text_list1.append(row["question_text"] + " " + row["reply_text"])
                score_list1.append(s / cnt)

    X_test_tokenized1 = tokenizer(text_list1, padding=True, truncation=True, max_length=512)
    X_test_tokenized2 = tokenizer(text_list2, padding=True, truncation=True, max_length=512)
    X_test_tokenized3 = tokenizer(text_list3, padding=True, truncation=True, max_length=512)

    # Create torch dataset
    test_dataset1 = Dataset(X_test_tokenized1)
    test_dataset2 = Dataset(X_test_tokenized2)
    test_dataset3 = Dataset(X_test_tokenized3)

    # Load trained model
    model_path = "out/output_" + cur_group + "/checkpoint-500"
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=1)

    # Define test trainer
    test_trainer = Trainer(model)

    # Make prediction
    raw_pred1, _, _ = test_trainer.predict(test_dataset1)
    raw_pred2, _, _ = test_trainer.predict(test_dataset2)
    raw_pred3, _, _ = test_trainer.predict(test_dataset3)
    
    tmp = []
    raw_pred1 = raw_pred1.reshape((len(raw_pred1),))
    raw_pred2 = raw_pred2.reshape((len(raw_pred2),))
    raw_pred3 = raw_pred3.reshape((len(raw_pred3),))

    tmp.append(np.corrcoef(raw_pred1, np.array(score_list1))[0,1])
    tmp.append(np.corrcoef(raw_pred2, np.array(score_list2))[0,1])
    tmp.append(np.corrcoef(raw_pred3, np.array(score_list3))[0,1])
    ret[cur_group] = tmp


  0%|          | 0/25 [00:00<?, ?it/s]loading configuration file out/output_group_01/checkpoint-500/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file out/output_group_01/checkpoint-500/pytorch_model.bin
All model checkpoint

***** Running Prediction *****
  Num examples = 77
  Batch size = 8
***** Running Prediction *****
  Num examples = 75
  Batch size = 8
  4%|▍         | 1/25 [00:10<04:16, 10.67s/it]loading configuration file out/output_group_02/checkpoint-500/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "type_vocab_size":

***** Running Prediction *****
  Num examples = 64
  Batch size = 8
***** Running Prediction *****
  Num examples = 59
  Batch size = 8
  8%|▊         | 2/25 [00:27<05:11, 13.56s/it]


OSError: Can't load config for 'out/output_group_03/checkpoint-500'. Make sure that:

- 'out/output_group_03/checkpoint-500' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'out/output_group_03/checkpoint-500' is not a path to a local directory with something else, in that case)

- or 'out/output_group_03/checkpoint-500' is the correct path to a directory containing a config.json file



In [58]:
ret

{'group_01': [0.5793006007257147, 0.755119156853009, -0.04148565743116314],
 'group_02': [0.5724079108338965, 0.7099946789968093, -0.24757961114789137]}